# Amazon Bedrock - Image Generation with Nova Canvas
This notebook demonstrates how to generate images using Amazon Nova Canvas model

In [ ]:
# %% Import necessary packages
import boto3
import json
import base64
from PIL import Image
from io import BytesIO
from datetime import datetime
import os

In [ ]:
# %% Initialize Bedrock Runtime client
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
model_id = "amazon.nova-canvas-v1:0"

In [ ]:
# %% Helper function to display and save images
def display_and_save_image(base64_string, filename=None):
    """Decode base64 image, display it, and optionally save to file"""
    image_bytes = base64.b64decode(base64_string)
    image = Image.open(BytesIO(image_bytes))
    
    # Display image
    display(image)
    
    # Save if filename provided
    if filename:
        os.makedirs('generated_images', exist_ok=True)
        filepath = f"generated_images/{filename}"
        image.save(filepath)
        print(f"Image saved to: {filepath}")
    
    return image

In [ ]:
# %% Basic text-to-image generation
prompt = "A modern yellow bulldozer working on a construction site at sunset, photorealistic, high detail"

request_body = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "height": 1024,
        "width": 1024,
        "cfgScale": 8.0,
        "seed": 42
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Text-to-Image Generation:")
print("=" * 80)
print(f"Prompt: {prompt}")
print(f"\nGenerated {len(response_body['images'])} image(s)")

# Display the generated image
for idx, image_data in enumerate(response_body['images']):
    filename = f"bulldozer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    display_and_save_image(image_data, filename)

In [ ]:
# %% Generate multiple images with different seeds
prompt = "Heavy-duty excavator digging in a quarry, industrial photography style"

print("Generating 3 variations:")
print("=" * 80)

for i in range(3):
    request_body = {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "standard",
            "height": 768,
            "width": 1280,
            "cfgScale": 7.0,
            "seed": 100 + i
        }
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(request_body)
    )
    
    response_body = json.loads(response['body'].read())
    
    print(f"\nVariation {i+1} (seed={100+i}):")
    filename = f"excavator_var{i+1}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    display_and_save_image(response_body['images'][0], filename)

In [ ]:
# %% Image-to-image generation (variation)
# First, load an existing image
input_image_path = "path/to/your/reference_image.jpg"  # Update this path

# For demo, create a simple colored image
demo_image = Image.new('RGB', (512, 512), color='blue')
buffer = BytesIO()
demo_image.save(buffer, format='PNG')
input_image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

prompt = "Transform this into a realistic construction site with heavy machinery"

request_body = {
    "taskType": "IMAGE_VARIATION",
    "imageVariationParams": {
        "text": prompt,
        "images": [input_image_base64],
        "similarityStrength": 0.7  # 0.2-1.0, higher = more similar to input
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "height": 512,
        "width": 512,
        "cfgScale": 8.0
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Image Variation:")
print("=" * 80)
print(f"Prompt: {prompt}")
print(f"Similarity Strength: 0.7")

filename = f"variation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
display_and_save_image(response_body['images'][0], filename)

In [ ]:
# %% Inpainting - Edit specific parts of an image
# Load base image and mask
# Mask: white areas will be regenerated, black areas preserved

# Demo: Create sample image and mask
base_image = Image.new('RGB', (512, 512), color='gray')
buffer = BytesIO()
base_image.save(buffer, format='PNG')
base_image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

# Create mask (white circle in center)
mask_image = Image.new('L', (512, 512), color='black')
from PIL import ImageDraw
draw = ImageDraw.Draw(mask_image)
draw.ellipse([156, 156, 356, 356], fill='white')
buffer = BytesIO()
mask_image.save(buffer, format='PNG')
mask_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

prompt = "A red forklift in a warehouse"

request_body = {
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": prompt,
        "image": base_image_base64,
        "maskImage": mask_base64
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 8.0
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Inpainting:")
print("=" * 80)
print(f"Prompt: {prompt}")
print("White areas in mask will be regenerated")

filename = f"inpaint_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
display_and_save_image(response_body['images'][0], filename)

In [ ]:
# %% Outpainting - Extend image beyond original boundaries
# Create a base image
base_image = Image.new('RGB', (512, 512), color='green')
buffer = BytesIO()
base_image.save(buffer, format='PNG')
base_image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

# Mask for outpainting (white = areas to generate)
mask_image = Image.new('L', (512, 512), color='black')
draw = ImageDraw.Draw(mask_image)
# Paint white borders (areas to extend)
draw.rectangle([0, 0, 511, 100], fill='white')  # Top
draw.rectangle([0, 412, 511, 511], fill='white')  # Bottom
buffer = BytesIO()
mask_image.save(buffer, format='PNG')
mask_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

prompt = "Extend this construction site scene with more equipment and landscape"

request_body = {
    "taskType": "OUTPAINTING",
    "outPaintingParams": {
        "text": prompt,
        "image": base_image_base64,
        "maskImage": mask_base64,
        "outPaintingMode": "DEFAULT"  # or "PRECISE"
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 8.0
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Outpainting:")
print("=" * 80)
print(f"Prompt: {prompt}")
print("Extending image in white mask areas")

filename = f"outpaint_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
display_and_save_image(response_body['images'][0], filename)

In [ ]:
# %% Background removal
# Load an image with a subject to isolate
input_image = Image.new('RGB', (512, 512), color='skyblue')
draw = ImageDraw.Draw(input_image)
draw.ellipse([156, 156, 356, 356], fill='orange')  # Simple subject
buffer = BytesIO()
input_image.save(buffer, format='PNG')
input_image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

request_body = {
    "taskType": "BACKGROUND_REMOVAL",
    "backgroundRemovalParams": {
        "image": input_image_base64
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Background Removal:")
print("=" * 80)
print("Subject isolated with transparent background")

filename = f"no_bg_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
display_and_save_image(response_body['images'][0], filename)

In [ ]:
# %% Conditional image generation with color control
prompt = "A mobile crane lifting steel beams"
negative_prompt = "blurry, low quality, distorted, cartoon"

request_body = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt,
        "negativeText": negative_prompt  # What to avoid
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "premium",  # standard or premium
        "height": 1024,
        "width": 1024,
        "cfgScale": 10.0,  # Higher = more adherence to prompt
        "seed": 12345
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Conditional Generation with Negative Prompt:")
print("=" * 80)
print(f"Prompt: {prompt}")
print(f"Negative Prompt: {negative_prompt}")
print(f"Quality: premium")
print(f"CFG Scale: 10.0")

filename = f"crane_premium_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
display_and_save_image(response_body['images'][0], filename)

In [ ]:
# %% Summary of parameters
print("Nova Canvas Image Generation Parameters:")
print("=" * 80)
print("\nTask Types:")
print("  - TEXT_IMAGE: Generate from text prompt")
print("  - IMAGE_VARIATION: Create variations of input image")
print("  - INPAINTING: Edit specific regions")
print("  - OUTPAINTING: Extend image boundaries")
print("  - BACKGROUND_REMOVAL: Remove background")
print("\nImage Sizes (width x height):")
print("  - 1024x1024, 768x1280, 1280x768")
print("  - 768x1408, 1408x768, 640x1536, 1536x640")
print("\nQuality:")
print("  - standard: Faster, lower cost")
print("  - premium: Higher quality, more detail")
print("\nCFG Scale: 1.0-10.0")
print("  - Lower: More creative/diverse")
print("  - Higher: More adherent to prompt")
print("\nSimilarity Strength (IMAGE_VARIATION): 0.2-1.0")
print("  - Lower: More variation from input")
print("  - Higher: Closer to input image")
print("\nSeed: Any integer (for reproducibility)")